**Table of contents**<a id='toc0_'></a>    
- [What is the set of ambiguous symbols per source?](#toc1_1_)    
    - [Define Functions](#toc1_1_1_)    
    - [ENSG](#toc1_1_2_)    
    - [HGNC](#toc1_1_3_)    
    - [NCBI](#toc1_1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[What is the set of ambiguous symbols per source?](#toc0_)

Need this to create a better histogram figure. Current one only shows the distribution of ambiguous symbols created by alias-alias collisions by source. I want to be able to illustrate how the ambiguous symbols in each source (created by alias-alias and alias-primary collisions) are shared. 

In [1]:
import pandas as pd
import plotly.express as px

### <a id='toc1_1_1_'></a>[Define Functions](#toc0_)

In [2]:
def combine_columns(df, columns_to_combine, columns_to_keep, new_name, columns_to_drop):
    """Combine multiple columns into one while keeping associated data attached.
    Use this function when the columns to combine are easier to list 
    than the columns not to combine.
    
    :param df: The DataFrame containing the columns to be combined
    :param columns_to_combine: List of column names to combine into one
    :param columns_to_keep: List of columns to keep in the final DataFrame
    :param new_name: The name of the new combined column
    :param columns_to_drop: List of columns to drop from the final DataFrame
    :return: A new DataFrame with combined columns and selected columns retained
    """
    og_df = df.copy()

    combined_dfs = []

    # Loop through each column in columns_to_combine and create a new DataFrame
    for col in columns_to_combine:
        temp_df = og_df[list(set([col] + columns_to_keep))].copy()
        temp_df[new_name] = temp_df[col]
        combined_dfs.append(temp_df)

    df_combined = pd.concat(combined_dfs, ignore_index=True)
    df_combined.drop(columns_to_drop, axis=1, inplace=True)
    df_combined.drop_duplicates(inplace=True)
    
    return df_combined

In [3]:
def create_ambiguous_symbol_histogram(XXXX_ambiguous_symbol_counts: pd.DataFrame, source: str, XXXX_ambiguous_symbol_set_count: int):
    """Create a histogram of the frequencies at which aliases are shared

    :param aa_collision_gene_xxxx_df: A df of alias-alias collisions organized by primary gene symbol
    :param source: Representation of the source of the gene records
    :param xxxx_alias_count: Number of aliases total in the source
    :return: A histogram of the percentage of aliases that are shared between 2 genes, 3 genes, and so on
    """

    #Count the frequency at which aliases are shared 
    XXXX_ambiguous_symbol_counts_distribution_df = XXXX_ambiguous_symbol_counts.pivot_table(
    index=["symbol_counts"], aggfunc="size"
    )
    XXXX_ambiguous_symbol_counts_distribution_df = XXXX_ambiguous_symbol_counts_distribution_df.reset_index()
    XXXX_ambiguous_symbol_counts_distribution_df.rename(columns={0: "num_collision_symbol"}, inplace=True)
    XXXX_ambiguous_symbol_counts_distribution_df["percent_collision_symbol"] = (
        XXXX_ambiguous_symbol_counts_distribution_df["num_collision_symbol"] / XXXX_ambiguous_symbol_set_count
    ) * 100

    #Convert to csv
    XXXX_ambiguous_symbol_counts_distribution_df.to_csv(f'../output/ambiguous_symbol_{source}_distribution_df.csv', index=True)

    #Create histogram df 
    xxxx_symbol_count_histogram_df = XXXX_ambiguous_symbol_counts_distribution_df.drop(
    "num_collision_symbol", axis=1)

    #Convert to csv
    xxxx_symbol_count_histogram_df.to_csv(f'../output/{source}_ambiguous_symbol_count_histogram_df.csv', index=True)

    return px.bar(xxxx_symbol_count_histogram_df, x="symbol_counts", y="percent_collision_symbol")



### <a id='toc1_1_2_'></a>[ENSG](#toc0_)

import file with ambiguous symbols resulting from alias-alias collisions

In [4]:
aa_collision_alias_ensg_df = pd.read_csv(
    "../output/aa_collision_alias_ensg_df.csv", index_col=[0])
aa_collision_alias_ensg_df

,collision,ENSG_ID,gene_symbol,source
0,2F1,"ENSG00000139187, ENSG00000005022","KLRG1, SLC25A5",ENSG
1,60B8AG,"ENSG00000143546, ENSG00000163220","S100A8, S100A9",ENSG
2,87U6,"ENSG00000065135, ENSG00000206832","GNAI3, RNU6V",ENSG
3,9G8,"ENSG00000115875, ENSG00000164609","SRSF7, SLU7",ENSG
4,A1,"ENSG00000163918, ENSG00000033627, ENSG00000049...","RFC4, ATP6V0A1, RFC2, RFC1",ENSG
...,...,...,...,...
1612,ZIP4,"ENSG00000120498, ENSG00000285243","TEX11, SLC39A4",ENSG
1613,ZNF422,"ENSG00000165512, ENSG00000172943","ZNF22, PHF8",ENSG
1614,ZNT8,"ENSG00000164756, ENSG00000196660","SLC30A8, SLC30A10",ENSG
1615,ZRC1,"ENSG00000196660, ENSG00000170385","SLC30A10, SLC30A1",ENSG


In [5]:
aa_collision_alias_ensg_set = set(
    aa_collision_alias_ensg_df["collision"]
)
len(aa_collision_alias_ensg_set)

1617

import file with ambiguous symbols resulting from alias-primary collisions

In [6]:
ap_collision_ensg_df = pd.read_csv(
    "../output/merged_alias_ap_collision_ensg_df.csv", index_col=[0])
ap_collision_ensg_df

,gene_symbol,alias_symbol,ENSG_ID,HGNC_ID,NCBI_ID,collision,source
0,RN7SK,7SK,ENSG00000283293,10037.0,125050,7SK,ENSG
1,SOAT1,"ACAT,ACAT1,SOAT,STAT",ENSG00000057252,11177.0,6646,ACAT1,ENSG
2,SOAT2,ACAT2,ENSG00000167780,11178.0,8435,ACAT2,ENSG
3,NDUFAB1,"ACP,ACP1,FASN2A,SDAP",ENSG00000004779,7694.0,4706,ACP1,ENSG
4,ACTBP8,ACTBP2,ENSG00000220267,141.0,0,ACTBP2,ENSG
...,...,...,...,...,...,...,...
689,ZNF121,"D19S204,ZHC32,ZNF20",ENSG00000197961,12904.0,7675,ZNF20,ENSG
690,RNF141,"ZFP26,ZNF230",ENSG00000110315,21159.0,50862,ZNF230,ENSG
691,ZNF322P1,"ZNF322,ZNF322B",ENSG00000188801,14003.0,0,ZNF322,ENSG
692,ZNF106,"SH3BP3,ZFP106,ZNF474",ENSG00000103994,12886.0,64397,ZNF474,ENSG


In [7]:
ap_collision_ensg_df.loc[ap_collision_ensg_df["collision"] == "SKI"]

,gene_symbol,alias_symbol,ENSG_ID,HGNC_ID,NCBI_ID,collision,source
575,HHAT,"FLJ10724,GUP2,MART-2,MART2,RASP,SIT,SKI,SKN",ENSG00000280680,18270.0,55733,SKI,ENSG


In [8]:
ap_collision_alias_ensg_set = set(
    ap_collision_ensg_df["collision"]
)
len(ap_collision_alias_ensg_set)

621

combine lists without duplicates to create list of ambiguous symbols for ENSG

In [9]:
ensg_ambiguous_symbol_set = aa_collision_alias_ensg_set.union(
    ap_collision_alias_ensg_set
)
ensg_ambiguous_symbol_set_count = len(ensg_ambiguous_symbol_set)
ensg_ambiguous_symbol_set_count

2179

import mini_ensg_df from symbol_capture_generation.ipynb

In [10]:
mini_ensg_df = pd.read_csv(
        ("../output/mini_ensg_df.csv"), index_col=[0],dtype={"NCBI_ID": str,"HGNC_ID":str})
ensg_df = mini_ensg_df[mini_ensg_df['gene_symbol'] != mini_ensg_df['alias_symbol']]
ensg_df['primary_gene_symbol'] = ensg_df['gene_symbol'].str.upper()
ensg_df.drop(["gene_symbol"], axis=1, inplace=True)
ensg_df['alias_symbol'] = ensg_df['alias_symbol'].str.upper()
ensg_df = ensg_df.drop_duplicates(subset=['primary_gene_symbol', 'alias_symbol'], keep='first')
ensg_df

/var/folders/vt/znzp_c6s02q6kjcmqfk0cb500000gq/T/ipykernel_37978/2002703901.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ensg_df['primary_gene_symbol'] = ensg_df['gene_symbol'].str.upper()
/var/folders/vt/znzp_c6s02q6kjcmqfk0cb500000gq/T/ipykernel_37978/2002703901.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ensg_df.drop(["gene_symbol"], axis=1, inplace=True)
/var/folders/vt/znzp_c6s02q6kjcmqfk0cb500000gq/T/ipykernel_37978/2002703901.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,ENSG_ID,alias_symbol,HGNC_ID,NCBI_ID,primary_gene_symbol
0,ENSG00000210049,MTTF,7481,NaN,MT-TF
1,ENSG00000210049,TRNF,7481,NaN,MT-TF
2,ENSG00000211459,12S,7470,NaN,MT-RNR1
3,ENSG00000211459,MOTS-C,7470,NaN,MT-RNR1
4,ENSG00000211459,MTRNR1,7470,NaN,MT-RNR1
...,...,...,...,...,...
117135,ENSG00000200033,NaN,47366,NaN,RNU6-403P
117136,ENSG00000228437,LNCSLCC1,53417,NaN,LINC02474
117137,ENSG00000228437,RP11-400N13.2,53417,NaN,LINC02474
117138,ENSG00000229463,LYST-IT2,41320,NaN,LYST-AS1


In [11]:
ensg_df.loc[ensg_df["alias_symbol"] == "MTTF"]

,ENSG_ID,alias_symbol,HGNC_ID,NCBI_ID,primary_gene_symbol
0,ENSG00000210049,MTTF,7481,NaN,MT-TF


make a general symbol column with primary and alias symbols

In [12]:
ensg_df = combine_columns(ensg_df, ["primary_gene_symbol", "alias_symbol"], ["primary_gene_symbol", "ENSG_ID", "HGNC_ID","NCBI_ID"], "symbol", "alias_symbol")
ensg_df

,ENSG_ID,NCBI_ID,primary_gene_symbol,HGNC_ID,symbol
0,ENSG00000210049,NaN,MT-TF,7481,MT-TF
2,ENSG00000211459,NaN,MT-RNR1,7470,MT-RNR1
5,ENSG00000210077,NaN,MT-TV,7500,MT-TV
7,ENSG00000210082,NaN,MT-RNR2,7471,MT-RNR2
10,ENSG00000209082,NaN,MT-TL1,7490,MT-TL1
...,...,...,...,...,...
147385,ENSG00000200033,NaN,RNU6-403P,47366,NaN
147386,ENSG00000228437,NaN,LINC02474,53417,LNCSLCC1
147387,ENSG00000228437,NaN,LINC02474,53417,RP11-400N13.2
147388,ENSG00000229463,NaN,LYST-AS1,41320,LYST-IT2


In [13]:
ensg_df.loc[ensg_df["primary_gene_symbol"] == "MT-TF"]

,ENSG_ID,NCBI_ID,primary_gene_symbol,HGNC_ID,symbol
0,ENSG00000210049,NaN,MT-TF,7481,MT-TF
73695,ENSG00000210049,NaN,MT-TF,7481,MTTF
73696,ENSG00000210049,NaN,MT-TF,7481,TRNF


use list of ambiguous symbols and ensg_combined_concept_ids_df to create a histogram of how many genes share ambiguous symbols

In [14]:
ensg_symbol_value_counts = ensg_df["symbol"].value_counts()

ensg_ambiguous_symbol_counts = ensg_symbol_value_counts[ensg_symbol_value_counts.index.isin(ensg_ambiguous_symbol_set)]
ensg_ambiguous_symbol_counts = ensg_ambiguous_symbol_counts.reset_index()
ensg_ambiguous_symbol_counts.columns = ["symbols", "symbol_counts"]
ensg_ambiguous_symbol_counts

,symbols,symbol_counts
0,MT1,11
1,HOX1,10
2,P40,10
3,P14,9
4,HOX2,9
...,...,...
2174,CYP2C,2
2175,C7,2
2176,EWSR2,2
2177,DSC2,2


In [15]:
create_ambiguous_symbol_histogram(ensg_ambiguous_symbol_counts, "ENSG", ensg_ambiguous_symbol_set_count)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'symbol_counts=%{x}<br>percent_collision_symbol=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': {'bdata': 'AgMEBQYHCAkKCw==', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('J7ttHn9CVUBhmCVAStglQMnI8VJq/Q' ... 'uBPH+3P8fMO4E8f7c/x8w7gTx/pz8='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'symbol_counts'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'percent_collision_symbol'}}}
})

### <a id='toc1_1_3_'></a>[HGNC](#toc0_)

import file with ambiguous symbols resulting from alias-alias collisions

In [16]:
aa_collision_alias_hgnc_df = pd.read_csv(
    "../output/aa_collision_alias_hgnc_df.csv", index_col=[0])
aa_collision_alias_hgnc_df

,collision,ENSG_ID,gene_symbol,source
0,2F1,"ENSG00000139187, ENSG00000005022","KLRG1, SLC25A5",HGNC
1,60B8AG,"ENSG00000143546, ENSG00000163220","S100A8, S100A9",HGNC
2,87U6,"ENSG00000206832, ENSG00000065135","RNU6V, GNAI3",HGNC
3,9G8,"ENSG00000115875, ENSG00000164609","SRSF7, SLU7",HGNC
4,A1,"ENSG00000033627, ENSG00000035928, ENSG00000163...","ATP6V0A1, RFC1, RFC4, RFC2",HGNC
...,...,...,...,...
1245,ZIP2,"ENSG00000165181, ENSG00000165794","SHOC1, SLC39A2",HGNC
1246,ZIP4,"ENSG00000120498, ENSG00000147804","TEX11, SLC39A4",HGNC
1247,ZNF422,"ENSG00000172943, ENSG00000165512","PHF8, ZNF22",HGNC
1248,ZNT8,"ENSG00000196660, ENSG00000164756","SLC30A10, SLC30A8",HGNC


In [17]:
aa_collision_alias_hgnc_set = set(
    aa_collision_alias_hgnc_df["collision"]
)
len(aa_collision_alias_hgnc_set)

1250

import file with ambiguous symbols resulting from alias-primary collisions

In [18]:
ap_collision_hgnc_df = pd.read_csv(
    "../output/merged_alias_ap_collision_hgnc_df.csv", index_col=[0])
ap_collision_hgnc_df

,gene_symbol,alias_symbol,ENSG_ID,HGNC_ID,NCBI_ID,collision,source
0,PPP1R12C,"DKFZP434D0412,p84,MBS85,p85,AAVS1",ENSG00000125503,14947,54776,AAVS1,HGNC
1,SOAT1,"ACAT,ACAT1",ENSG00000057252,11177,6646,ACAT1,HGNC
2,SOAT2,ACAT2,ENSG00000167780,11178,8435,ACAT2,HGNC
3,GLI3,"PAP-A,PAPA,PAPA1,PAPB,ACLS,PPDIV",ENSG00000106571,4319,2737,ACLS,HGNC
4,NDUFAB1,"SDAP,FASN2A,ACP,ACP1",ENSG00000004779,7694,4706,ACP1,HGNC
...,...,...,...,...,...,...,...
648,ZNF121,"ZHC32,ZNF20",ENSG00000197961,12904,7675,ZNF20,HGNC
649,RNF141,"ZFP26,ZNF230",ENSG00000110315,21159,50862,ZNF230,HGNC
650,ZNF106,"ZNF474,SH3BP3",ENSG00000103994,12886,64397,ZNF474,HGNC
651,ZFP1,"FLJ34243,ZNF475",ENSG00000184517,23328,162239,ZNF475,HGNC


In [19]:
ap_collision_alias_hgnc_set = set(
    ap_collision_hgnc_df["collision"]
)
len(ap_collision_alias_hgnc_set)

576

combine lists without duplicates to create list of ambiguous symbols for HGNC

In [20]:
hgnc_ambiguous_symbol_set = aa_collision_alias_hgnc_set.union(
    ap_collision_alias_hgnc_set
)
hgnc_ambiguous_symbol_set_count = len(hgnc_ambiguous_symbol_set)
hgnc_ambiguous_symbol_set_count

1758

import mini_hgnc_df from symbol_capture_generation.ipynb

In [21]:
mini_hgnc_df = pd.read_csv(
        ("../output/mini_hgnc_df.csv"), index_col=[0],dtype={"NCBI_ID": str,"HGNC_ID":str})
hgnc_df = mini_hgnc_df[mini_hgnc_df['gene_symbol'] != mini_hgnc_df['alias_symbol']]
hgnc_df['primary_gene_symbol'] = hgnc_df['gene_symbol'].str.upper()
hgnc_df.drop(["gene_symbol"], axis=1, inplace=True)
hgnc_df['alias_symbol'] = hgnc_df['alias_symbol'].str.upper()
hgnc_df = hgnc_df.drop_duplicates(subset=['primary_gene_symbol', 'alias_symbol'], keep='first')
hgnc_df

,HGNC_ID,alias_symbol,NCBI_ID,ENSG_ID,primary_gene_symbol
0,5,NaN,1,ENSG00000121410,A1BG
1,37133,FLJ23569,503538,ENSG00000268895,A1BG-AS1
2,24086,ACF,29974,ENSG00000148584,A1CF
3,24086,ASP,29974,ENSG00000148584,A1CF
4,24086,ACF64,29974,ENSG00000148584,A1CF
...,...,...,...,...,...
67578,29027,KIAA0399,23140,ENSG00000074755,ZZEF1
67579,29027,ZZZ4,23140,ENSG00000074755,ZZEF1
67580,29027,FLJ10821,23140,ENSG00000074755,ZZEF1
67581,24523,DKFZP564I052,26009,ENSG00000036549,ZZZ3


make a general column with primary and alias symbols

In [22]:
hgnc_df = combine_columns(hgnc_df, ["primary_gene_symbol", "alias_symbol"], ["primary_gene_symbol", "ENSG_ID", "HGNC_ID","NCBI_ID"], "symbol", "alias_symbol")
hgnc_df

,ENSG_ID,NCBI_ID,primary_gene_symbol,HGNC_ID,symbol
0,ENSG00000121410,1,A1BG,5,A1BG
1,ENSG00000268895,503538,A1BG-AS1,37133,A1BG-AS1
2,ENSG00000148584,29974,A1CF,24086,A1CF
7,ENSG00000175899,2,A2M,7,A2M
10,ENSG00000245105,144571,A2M-AS1,27057,A2M-AS1
...,...,...,...,...,...
135093,ENSG00000074755,23140,ZZEF1,29027,KIAA0399
135094,ENSG00000074755,23140,ZZEF1,29027,ZZZ4
135095,ENSG00000074755,23140,ZZEF1,29027,FLJ10821
135096,ENSG00000036549,26009,ZZZ3,24523,DKFZP564I052


use list of ambiguous symbols and hgnc_combined_concept_ids_df to create a histogram of how many genes share ambiguous symbols

In [23]:
hgnc_symbol_value_counts = hgnc_df["symbol"].value_counts()

hgnc_ambiguous_symbol_counts = hgnc_symbol_value_counts[hgnc_symbol_value_counts.index.isin(hgnc_ambiguous_symbol_set)]
hgnc_ambiguous_symbol_counts = hgnc_ambiguous_symbol_counts.reset_index()
hgnc_ambiguous_symbol_counts.columns = ["symbols", "symbol_counts"]
hgnc_ambiguous_symbol_counts

,symbols,symbol_counts
0,P40,10
1,P14,9
2,U3,9
3,PAP,7
4,U4,7
...,...,...
1753,BCRP1,2
1754,HHZ150,2
1755,PC,2
1756,SNX30,2


In [24]:
create_ambiguous_symbol_histogram(hgnc_ambiguous_symbol_counts, "HGNC", hgnc_ambiguous_symbol_set_count)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'symbol_counts=%{x}<br>percent_collision_symbol=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': {'bdata': 'AgMEBQYHCQo=', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('9oykZQ3OVEDt3Vq4aTsoQJUeWRLP1w' ... 'O+H80/G34hw74fvT8bfiHDvh+tPw=='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'symbol_counts'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'percent_collision_symbol'}}}
})

### <a id='toc1_1_4_'></a>[NCBI](#toc0_)

import file with ambiguous symbols resulting from alias-alias collisions

In [25]:
aa_collision_alias_ncbi_df = pd.read_csv(
    "../output/aa_collision_alias_ncbi_df.csv", index_col=[0])
aa_collision_alias_ncbi_df

,collision,ENSG_ID,gene_symbol,source
0,10Q23DEL,"ENSG00000171862, ENSG00000107779","PTEN, BMPR1A",NCBI
1,12-LOX,"ENSG00000161905, ENSG00000108839","ALOX15, ALOX12",NCBI
2,20-ALPHA-HSD,"ENSG00000187134, ENSG00000108786","AKR1C1, HSD17B1",NCBI
3,2F1,"ENSG00000005022, ENSG00000139187","SLC25A5, KLRG1",NCBI
4,3-ALPHA-HSD,"ENSG00000198610, ENSG00000073737","AKR1C4, DHRS9",NCBI
...,...,...,...,...
3668,ZNF769,"ENSG00000272968, ENSG00000146587","RBAK-RBAKDN, RBAK",NCBI
3669,ZNT8,"ENSG00000196660, ENSG00000164756","SLC30A10, SLC30A8",NCBI
3670,ZRC1,"ENSG00000170385, ENSG00000196660","SLC30A1, SLC30A10",NCBI
3671,ZWS,"ENSG00000127980, ENSG00000162928","PEX1, PEX13",NCBI


In [26]:
aa_collision_alias_ncbi_set = set(
    aa_collision_alias_ncbi_df["collision"]
)
len(aa_collision_alias_ncbi_set)

3673

import file with ambiguous symbols resulting from alias-primary collisions

In [27]:
ap_collision_ncbi_df = pd.read_csv(
    "../output/merged_alias_ap_collision_ncbi_df.csv", index_col=[0])
ap_collision_ncbi_df

,gene_symbol,alias_symbol,ENSG_ID,HGNC_ID,NCBI_ID,collision,source
0,IGHA2,A2M,ENSG00000211890,5479.0,3494,A2M,NCBI
1,NPSR1-AS1,AAA1,ENSG00000197085,22128.0,404744,AAA1,NCBI
2,CFAP91,"AAT1,AAT1alpha,C3orf15,CaM-IP2,MAATS1,SPATA26,...",ENSG00000183833,24010.0,89876,AAT1,NCBI
3,GPT,"AAT1,ALT,ALT1,GPT1,SGPT",ENSG00000167701,4552.0,2875,AAT1,NCBI
4,PPP1R12C,"AAVS1,LENG3,MBS85,p84,p85",ENSG00000125503,14947.0,54776,AAVS1,NCBI
...,...,...,...,...,...,...,...
2119,ZNF785,ZNF688,ENSG00000197162,26496.0,146540,ZNF688,NCBI
2120,ZP4,"ZBP,ZP1,ZP1B,ZPB,ZPB2,Zp-4",ENSG00000116996,15770.0,57829,ZP1,NCBI
2121,LMBR1,"ACHP,C7orf2,DIF14,LSS,PPD2,THYP,TPT,TPTPS,ZRS",ENSG00000105983,13243.0,64327,ZRS,NCBI
2122,ZNF446,"ZKSCAN20,ZSCAN30,ZSCAN52",ENSG00000083838,21036.0,55663,ZSCAN30,NCBI


In [28]:
ap_collision_alias_ncbi_set = set(
    ap_collision_ncbi_df["collision"]
)
len(ap_collision_alias_ncbi_set)

1566

combine lists without duplicates to create list of ambiguous symbols for NCBI

In [29]:
ncbi_ambiguous_symbol_set = aa_collision_alias_ncbi_set.union(
    ap_collision_alias_ncbi_set
)
ncbi_ambiguous_symbol_set_count = len(ncbi_ambiguous_symbol_set)
ncbi_ambiguous_symbol_set_count

5003

import mini_ncbi_df from symbol_capture_generation.ipynb

In [30]:
mini_ncbi_df = pd.read_csv(
        ("../output/mini_ncbi_df.csv"), index_col=[0],dtype={"NCBI_ID": str,"HGNC_ID":str})
ncbi_df = mini_ncbi_df[mini_ncbi_df['gene_symbol'] != mini_ncbi_df['alias_symbol']]
ncbi_df['primary_gene_symbol'] = ncbi_df['gene_symbol'].str.upper()
ncbi_df.drop(["gene_symbol"], axis=1, inplace=True)
ncbi_df['alias_symbol'] = ncbi_df['alias_symbol'].str.upper()
ncbi_df = ncbi_df.drop_duplicates(subset=['primary_gene_symbol', 'alias_symbol'], keep='first')
ncbi_df

,NCBI_ID,alias_symbol,HGNC_ID,ENSG_ID,primary_gene_symbol
0,1,A1B,5,ENSG00000121410,A1BG
0,1,ABG,5,ENSG00000121410,A1BG
0,1,GAB,5,ENSG00000121410,A1BG
0,1,HYST2477,5,ENSG00000121410,A1BG
1,2,A2MD,7,ENSG00000175899,A2M
...,...,...,...,...,...
193416,6775095,-,NaN,NaN,TRNQ
193417,6775096,-,NaN,NaN,TRNM
193418,6775097,-,NaN,NaN,TRNI
193419,6775098,-,NaN,NaN,ND1


make a general symbol column with primary and alias symbols

In [31]:
ncbi_df = combine_columns(ncbi_df, ["primary_gene_symbol", "alias_symbol"], ["primary_gene_symbol", "ENSG_ID", "HGNC_ID","NCBI_ID"], "symbol", "alias_symbol")
ncbi_df

,ENSG_ID,NCBI_ID,primary_gene_symbol,HGNC_ID,symbol
0,ENSG00000121410,1,A1BG,5,A1BG
4,ENSG00000175899,2,A2M,7,A2M
8,ENSG00000291190,3,A2MP1,8,A2MP1
9,ENSG00000171428,9,NAT1,7645,NAT1
13,ENSG00000156006,10,NAT2,7646,NAT2
...,...,...,...,...,...
183569,NaN,6775095,TRNQ,NaN,-
183570,NaN,6775096,TRNM,NaN,-
183571,NaN,6775097,TRNI,NaN,-
183572,NaN,6775098,ND1,NaN,-


use list of ambiguous symbols and ncbi_combined_concept_ids_df to create a histogram of how many genes share ambiguous symbols

In [32]:
ncbi_symbol_value_counts = ncbi_df["symbol"].value_counts()

ncbi_ambiguous_symbol_counts = ncbi_symbol_value_counts[ncbi_symbol_value_counts.index.isin(ncbi_ambiguous_symbol_set)]
ncbi_ambiguous_symbol_counts = ncbi_ambiguous_symbol_counts.reset_index()
ncbi_ambiguous_symbol_counts.columns = ["symbols", "symbol_counts"]
ncbi_ambiguous_symbol_counts

,symbols,symbol_counts
0,VH,37
1,H4C6,14
2,H4C15,14
3,H4C12,14
4,H4C14,14
...,...,...
4998,MIG-2,2
4999,RWDD4A,2
5000,CYSRS,2
5001,PKR2,2


In [33]:
create_ambiguous_symbol_histogram(ncbi_ambiguous_symbol_counts, "NCBI", ncbi_ambiguous_symbol_set_count)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'symbol_counts=%{x}<br>percent_collision_symbol=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': {'bdata': 'AgMEBQYHCAkKCwwOJQ==', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('ULiKW2l/VEByzRMXV04nQHe4Iedxqg' ... '50vHekP2/2kK1AMNM/uuTedLx3lD8='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'symbol_counts'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'percent_collision_symbol'}}}
})

In [34]:
ncbi_ensg_ambiguous_symbol_set = ncbi_ambiguous_symbol_set.union(
    ensg_ambiguous_symbol_set
)

In [35]:
ncbi_ensg_hgnc_ambiguous_symbol_set = ncbi_ensg_ambiguous_symbol_set.union(
    hgnc_ambiguous_symbol_set
)

In [36]:
ambiguous_symbol_set = ncbi_ensg_hgnc_ambiguous_symbol_set
len(ambiguous_symbol_set)

5172